# ⚡ Day 2 · Global Energy Mix
We keep the same rhythm: tiny concept bursts followed by hands-on code and immediate diagnostics. Today's question—*Are renewables scaling fast enough?*

## 🔄 How to Use This Solution
- Reuse the utilities for consistency across the week.
- Pause on the diagnostic cells to interpret slopes, units, and shapes before moving forward.
- Use optional extension prompts to differentiate for fast finishers.

> ### 🗂️ Data Card — Our World in Data Renewable Share
> - **Source:** Our World in Data compilation of BP Statistical Review & IEA datasets.
> - **Temporal coverage:** 1965–2022 (latest common year across technologies).
> - **Metrics:** Share of primary energy from renewables overall plus hydro, wind, and solar (% of total).
> - **Refresh cadence:** Annual; extracted January 2024.
> - **Caveats:** Shares are relative to primary energy; they understate distributed generation and do not capture efficiency gains.
> - **Ethics & framing:** Avoid implying causation between adoption and emissions cuts; note that energy demand is also rising.

In [ ]:

from __future__ import annotations
from pathlib import Path
from typing import Mapping, Sequence

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

DATA_DIR = Path.cwd() / "data"
PLOTS_DIR = Path.cwd() / "plots"
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

sns.set_theme(style="whitegrid", context="notebook", palette="colorblind")
plt.rcParams.update({
    "figure.dpi": 120,
    "axes.titlesize": 18,
    "axes.labelsize": 13,
    "axes.titleweight": "semibold",
    "axes.grid": True,
})


def baseline_style() -> None:
    """Reset plot style to the shared course defaults."""
    sns.set_theme(style="whitegrid", context="notebook", palette="colorblind")
    plt.rcParams.update({
        "axes.grid": True,
        "axes.spines.top": False,
        "axes.spines.right": False,
        "figure.dpi": 120,
        "font.size": 12,
    })


def load_data(path: Path, *, read_kwargs: Mapping[str, object] | None = None) -> pd.DataFrame:
    read_kwargs = dict(read_kwargs or {})
    df = pd.read_csv(path, **read_kwargs)
    print(f"✅ Loaded {path.name} with shape {df.shape}")
    return df


def validate_columns(df: pd.DataFrame, required: Sequence[str]) -> None:
    missing = [col for col in required if col not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")
    print("✅ Column check passed:", ", ".join(required))


def expect_rows_between(df: pd.DataFrame, lower: int, upper: int) -> None:
    rows = len(df)
    if not (lower <= rows <= upper):
        raise ValueError(f"Expected between {lower} and {upper} rows, got {rows}")
    print(f"✅ Row count within expected range ({rows} rows)")


def quick_diagnose(df: pd.DataFrame, *, label: str = "Data preview", n: int = 5) -> None:
    print(f"
🔍 {label}")
    display(df.head(n))
    numeric = df.select_dtypes(include="number")
    if not numeric.empty:
        display(numeric.describe().T)
    nulls = df.isna().sum()
    if nulls.any():
        print("⚠️ Null values detected:
", nulls[nulls > 0])
    else:
        print("✅ No null values detected in this slice.")


def accessibility_check(ax: plt.Axes) -> None:
    title_ok = bool(ax.get_title())
    label_ok = bool(ax.get_xlabel()) and bool(ax.get_ylabel())
    if not (title_ok and label_ok):
        raise ValueError("Add a descriptive title and axis labels before proceeding.")
    xlabels = [tick.get_text() for tick in ax.get_xticklabels()]
    if len(xlabels) > 12:
        ax.tick_params(axis='x', labelrotation=35)
    print("✅ Accessibility check: title, labels, and readable ticks confirmed.")


def annotate_latest_point(ax: plt.Axes, x: float, y: float, text: str) -> None:
    ax.scatter([x], [y], color=ax.lines[0].get_color(), s=60, zorder=5)
    ax.annotate(
        text,
        xy=(x, y),
        xytext=(0.96, 0.85),
        textcoords="axes fraction",
        ha="right",
        arrowprops={"arrowstyle": "->", "color": "#333"},
        fontsize=11,
    )


def require_story_elements(story: Mapping[str, str]) -> None:
    required = ["claim", "evidence", "visual", "takeaway", "source", "units"]
    missing = [key for key in required if not story.get(key, "").strip()]
    if missing:
        raise ValueError(f"Fill in the storytelling scaffold: missing {missing}")
    print("✅ Story scaffold complete.")


def save_last_fig(filename: str, fig: plt.Figure | None = None) -> None:
    fig = fig or plt.gcf()
    if not fig.axes:
        raise ValueError("No Matplotlib figure found to save.")
    output_path = PLOTS_DIR / filename
    fig.savefig(output_path, bbox_inches="tight")
    print(f"💾 Figure saved to {output_path.relative_to(Path.cwd())}")


## Loop 1 · Load Harmonised Tables
Goal: load each technology slice and isolate the `World` aggregate so we can combine them cleanly.

✅ **You should see:** Four dataframes with identical year ranges and no missing world-level rows.

In [ ]:

baseline_style()

renewable_total = load_data(DATA_DIR / "01 renewable-share-energy.csv")
hydro = load_data(DATA_DIR / "06 hydro-share-energy.csv")
wind = load_data(DATA_DIR / "10 wind-share-energy.csv")
solar = load_data(DATA_DIR / "14 solar-share-energy.csv")

world_total = renewable_total.query("Entity == 'World'").copy()
world_hydro = hydro.query("Entity == 'World'").copy()
world_wind = wind.query("Entity == 'World'").copy()
world_solar = solar.query("Entity == 'World'").copy()

for frame in (world_total, world_hydro, world_wind, world_solar):
    validate_columns(frame, ["Entity", "Code", "Year"])
    expect_rows_between(frame, 50, 70)

quick_diagnose(world_total.tail(), label="World renewables (tail)")


## Loop 2 · Merge & Sanity Check
Goal: build a tidy dataframe with aligned years and verify that technology shares sum close to the total.

✅ **You should see:** A combined dataframe with columns `total`, `hydro`, `wind`, `solar` and a small residual gap (<0.5 percentage points).

In [ ]:

merged = (
    world_total[["Year", "Renewables (% equivalent primary energy)"]]
    .rename(columns={"Renewables (% equivalent primary energy)": "total"})
    .merge(
        world_hydro[["Year", "Hydro (% equivalent primary energy)"]].rename(columns={"Hydro (% equivalent primary energy)": "hydro"}),
        on="Year",
    )
    .merge(
        world_wind[["Year", "Wind (% equivalent primary energy)"]].rename(columns={"Wind (% equivalent primary energy)": "wind"}),
        on="Year",
    )
    .merge(
        world_solar[["Year", "Solar (% equivalent primary energy)"]].rename(columns={"Solar (% equivalent primary energy)": "solar"}),
        on="Year",
    )
)

validate_columns(merged, ["Year", "total", "hydro", "wind", "solar"])
expect_rows_between(merged, 50, 70)
merged["residual"] = merged["total"] - merged[["hydro", "wind", "solar"]].sum(axis=1)
print("Residual summary (total - components):")
print(merged["residual"].describe()[["mean", "min", "max"]])
quick_diagnose(merged.tail(), label="Combined renewable shares")


## Loop 3 · Trend Plot with Guardrails
Goal: visualise total renewable share over time with milestones and built-in interpretation prompts.

✅ **You should see:** Smooth growth after 2000, with annotations flagging the latest value and Paris Agreement year.

In [ ]:

latest_year = int(merged["Year"].max())
latest_share = merged.loc[merged["Year"] == latest_year, "total"].item()
paris_share = merged.loc[merged["Year"] == 2015, "total"].item()

story = {
    "claim": "Renewables have tripled their share of global energy since 2000 but remain a modest slice of demand.",
    "evidence": f"Global renewable share rose from {merged.loc[merged['Year']==2000, 'total'].item():.1f}% in 2000 to {latest_share:.1f}% in {latest_year}.",
    "visual": "Line chart of global renewable energy share with milestone annotations.",
    "takeaway": "Progress is accelerating, yet 85% of energy still comes from other sources—pace must increase.",
    "source": "Source: Our World in Data (BP/IEA), downloaded Jan 2024.",
    "units": "Share of primary energy (%)",
}
require_story_elements(story)

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(merged["Year"], merged["total"], linewidth=2.5)
ax.set_title("Renewables Are Rising but Still a Minority of Global Energy")
ax.set_xlabel("Year")
ax.set_ylabel("Share of primary energy (%)")
ax.text(
    0.01,
    0.04,
    story["source"],
    transform=ax.transAxes,
    fontsize=9,
    color="#555",
)
annotate_latest_point(ax, latest_year, latest_share, f"{latest_year}: {latest_share:.1f}%")
ax.annotate(
    "2015 Paris Agreement",
    xy=(2015, paris_share),
    xytext=(2000, paris_share + 5),
    arrowprops={"arrowstyle": "->", "color": "#333"},
    fontsize=10,
)
accessibility_check(ax)
last_fig = fig
plt.show()

display(pd.DataFrame([story]).T.rename(columns={0: "Story Scaffold"}))


## Loop 4 · Composition Mini-Project
Stretch students with an optional stacked area chart that decomposes hydro, wind, and solar. Encourage them to discuss which technology drives growth post-2010.

In [ ]:

component_cols = ["hydro", "wind", "solar"]
fig, ax = plt.subplots(figsize=(12, 6))
ax.stackplot(
    merged["Year"],
    *[merged[col] for col in component_cols],
    labels=[col.title() for col in component_cols],
    alpha=0.7,
)
ax.set_title("Hydro Still Dominates, but Wind and Solar Drive Recent Gains")
ax.set_xlabel("Year")
ax.set_ylabel("Share of primary energy (%)")
ax.legend(loc="upper left")
ax.text(0.01, 0.04, "Source: Our World in Data (BP/IEA)", transform=ax.transAxes, fontsize=9, color="#555")
accessibility_check(ax)
plt.show()


In [ ]:
save_last_fig("day02_solution_plot.png", fig=last_fig)